In [1]:
import dataset3

In [2]:
data3 = dataset3.load_data(1)

In [3]:
import rec3


ae

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


import pandas as pd



In [5]:
%matplotlib inline
import matplotlib.pyplot as plt

In [9]:
def weighted_average(input, targets, weights):
    return nn.functional.mse_loss(input.mul(weights), targets.mul(weights),size_average=False)*1.0/weights.sum()

In [4]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.representations = dict()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out

NameError: name 'nn' is not defined

In [7]:
input_size = 6040
hidden_size = 200
output_size = 6040
learning_rate = 0.001
cf_flag=False


num_epochs = 15

cv = 1



In [3]:
representations = dict()
loss_history = list()

train_precision = dict()
test_precision = dict()

train_cf_precision = dict()
test_cf_precision = dict()

train_mse = dict()
test_mse = dict()


net = Net(input_size, hidden_size, output_size)
# net = nn.DataParallel(net)

certeria = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
print(net)
net.cuda()
for epoch in range(num_epochs):
    for i_batch, sample_batched in enumerate(data3.train_loader):
        net.train()
        id_batch = sample_batched['id']
        sample = Variable(sample_batched['sample']).cuda()
        groud_truth = Variable(torch.from_numpy(data3.train_dataset.known.T[id_batch.numpy()])).cuda()

#         dropout_index = 
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(sample)
        loss = certeria(outputs, groud_truth)
#         print(i_batch, loss.data[0])
        loss.backward()
        optimizer.step()
        
        print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   % (epoch + 1, num_epochs, i_batch + 1, 
                      len(data3.train_loader) ,loss.data[0]))
        loss_history.append(loss.data[0])
    iters = epoch +1
    if (iters) % 1 == 0:
        print ('_'*60)

        print('iteration %s:'%iters)
        name = 'ae-%s-epoch_%s_cv_%s'%(str(net).replace('\n',''), epoch+1, cv)
        loss_df= pd.DataFrame(loss_history,columns=['loss'])
        loss_df.plot(title=name)
        plt.show()

        net.eval()
        # reconstruct 
        print("reconstructing...")

        input_matrix = Variable(torch.from_numpy(data3.train_dataset.known.T)).cuda()
        reconstructed = net(input_matrix).cpu()
        # copy representations
        representations[iters] = dict()
        for key in net.representations.keys():
            representations[iters][key] = net.representations[key].copy()
#         print((reconstructed.size()))
        print('computing mse...')
        test_user_mask = np.zeros_like(data3.train_dataset.known)
        test_user_mask[data3.test_dataset.targets,:] = 1
        test_user_mask = Variable(torch.from_numpy(test_user_mask))
        train_mse[iters] = weighted_average(reconstructed, 
                                            Variable(torch.from_numpy(data3.train_dataset.known)),
                                            (1-test_user_mask)).data.numpy()[0]
        test_mse[iters] = weighted_average(reconstructed, 
                                           Variable(torch.from_numpy(data3.train_dataset.known)),
                                           (test_user_mask)).data.numpy()[0]
#         print((reconstructed.size()))

        print('prediction, computing precision...')
        reconstructed = reconstructed.data.numpy()
        reconstructed = reconstructed.T
#         print((reconstructed.shape))

        train_rec = rec_pred(reconstructed, train = np.zeros_like(data3.train_dataset.known), test = data3.train_dataset.known, 
                             targets = data3.train_dataset.train_users)
        test_rec = rec_pred(reconstructed, train = data3.train_dataset.known, test = data3.test_dataset.test, 
                             targets = data3.test_dataset.targets)
        train_precision[iters] = (train_rec.precision_, train_rec.recall_)
        test_precision[iters] = (test_rec.precision_, test_rec.recall_)
    
        if cf_flag:
        # CF on representations
            print ("CF on representations...")
            test_cf_precision[iters] = dict()
            for key in representations[iters].keys():
    #             for sim in ['dot']:
                for sim in ['dot','asymcos','cosine']:
                    cf = rec3.IBCF(sim=sim)
                    cf.fit(train_ratings=data3.train_dataset.train, profile = representations[iters][key])

    #                 for knn in [5]:
                    for knn in [5,50,100,200,500, 1000]:
                            print ("CF (%s,%s,%s) on representations..."%((key,sim,knn)))

                            cf.compute_score(input_ratings = data3.train_dataset.newusers, topN = knn ,
                                                     targets=data3.test_dataset.targets)
                            cf.produce_reclist(targets=data3.test_dataset.targets)
                            tmp_cf_perf = cf.evaluate(test=data3.test_dataset.test, rec_len = 5)
                            test_cf_precision[iters][key,sim,knn] = ( tmp_cf_perf)

        print("train_precision:")
        pprint(train_precision)
        print("test_precision" )
        pprint(test_precision)
        
        if cf_flag:
            print("test_cf_precision:")
            pprint(test_cf_precision)

        print("train_mse:")
        pprint(train_mse)
        print("test_mse" )
        pprint(test_mse)

        print ('_'*60)


NameError: name 'Net' is not defined

In [2]:
import sys
sys.getrefcount()

In [6]:
net = Net(input_size, hidden_size, output_size)


NameError: name 'Net' is not defined